In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
SOURCE_DIR = 'gdrive/MyDrive/NLP/ClassWork'

In [3]:
import torch
import re
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv(f"{SOURCE_DIR}/mahsa_amini_data.csv" )
df

,Unnamed: 0,Datetime,Text,PureText,Language,Sentiment,Date
0,0,2022-09-22 09:14:35+00:00,بنشین تا شود نقش فال ما \nنقش هم‌ فردا شدن\n#م...,بنشین تا شود نقش فال ما نقش هم‌ فردا شدن,fa,negative,2022-09-22
1,1,2022-10-06 01:44:55+00:00,@Tanasoli_Return @dr_moosavi این گوزو رو کی گر...,این گوزو رو کی گردن میگیره؟؟ دچار زوال عقل شده...,fa,very negative,2022-10-06
2,2,2022-09-22 15:12:28+00:00,@ghazaleghaffary برای ایران، برای مهسا.\n#OpIr...,برای ایران، برای مهسا.,fa,positive,2022-09-22
3,3,2022-09-22 09:35:50+00:00,@_hidden_ocean مرگ بر دیکتاتور \n#OpIran \n#Ma...,مرگ بر دیکتاتور,fa,very negative,2022-09-22
4,4,2022-09-22 01:31:25+00:00,نذاریم خونشون پایمال شه.‌‌.‌‌.\n#Mahsa_Amini #...,نذاریم خونشون پایمال شه.‌‌.‌‌.,fa,negative,2022-09-22
...,...,...,...,...,...,...,...
19995,19995,2022-09-24 13:21:44+00:00,برای ایران بانو #Mahsa_Amini #MahsaAmini ...,برای ایران بانو,fa,positive,2022-09-24
19996,19996,2022-09-30 10:28:32+00:00,@MohammadTehra16 @mimpedram از بس حاج خانم درا...,از بس حاج خانم دراز نشده واسش عقده دراز داره😅,fa,negative,2022-09-30
19997,19997,2022-10-07 21:22:19+00:00,به افتخار از بین رفتن جمهوری اسلامی🙆‍♂️🙆‍♂️🙆‍♂...,به افتخار از بین رفتن جمهوری اسلامی🙆‍♂️🙆‍♂️🙆‍♂...,fa,positive,2022-10-07
19998,19998,2022-09-22 17:27:40+00:00,پنجاه و شیش \n\n#مهسا_امینی \n#Mahsa_Amini \n#...,پنجاه و شیش,fa,no sentiment expressed,2022-09-22


In [ ]:
df["Sentiment"].unique()

array(['negative', 'very negative', 'positive', 'no sentiment expressed',
       'very positive', 'mixed'], dtype=object)

In [ ]:
df.iloc[0]["Text"]

'بنشین تا شود نقش فال ما \nنقش هم\u200c فردا شدن\n#مهسا_امینی \n#Mahsa_Amini \n#MahsaAmini'

In [ ]:
cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)

In [5]:
def find_k_nearest_neighbors(word, embedding_dict, k):
  words_cosine_similarity = dict()
  for token in embedding_dict.keys():
    words_cosine_similarity[token] = cos(embedding_dict[word], embedding_dict[token]).item()
  words_cosine_similarity = dict(sorted(words_cosine_similarity.items(), key=lambda item: item[1]))
  return list(words_cosine_similarity.keys())[-k:][::-1]

def delete_hashtag_usernames(text):
  try:
    result = []
    for word in text.split():
      if word[0] not in ['@', '#']:
        result.append(word)
    return ' '.join(result)
  except:
    return ''

def delete_url(text):
  text = re.sub(r'http\S+', '', text)
  return text

In [6]:
df["Text"].apply(delete_hashtag_usernames).apply(delete_url)

0                 بنشین تا شود نقش فال ما نقش هم‌ فردا شدن
1        این گوزو رو کی گردن میگیره؟؟ دچار زوال عقل شده...
2                                   برای ایران، برای مهسا.
3                                          مرگ بر دیکتاتور
4                           نذاریم خونشون پایمال شه.‌‌.‌‌.
                               ...                        
19995                                     برای ایران بانو 
19996        از بس حاج خانم دراز نشده واسش عقده دراز داره😅
19997    به افتخار از بین رفتن جمهوری اسلامی🙆‍♂️🙆‍♂️🙆‍♂...
19998                                          پنجاه و شیش
19999    در محیط طوفان‌زای ماهرانه در جنگ است ناخدای اس...
Name: Text, Length: 20000, dtype: object

In [7]:
word = 'زندگی'
k = 10

#0. Data preprocessing

In [ ]:
!pip install json-lines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json_lines

In [8]:
# 1. extract all tweets from files and save them in memory base on each year.
pure_text_list = df["Text"].apply(delete_hashtag_usernames).apply(delete_url).tolist()
# 2. remove urls, hashtags and usernames.

In [9]:
unique_words = set()

for sentence in pure_text_list:
  words = sentence.split()
  for word in words:
    unique_words.add(word)
unique_words = sorted(list(unique_words))

In [ ]:
unique_words.index("زندگی")

14913

#1. One hot encoding

In [ ]:
np.zeros(10)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
dict_one_hot = {}
for i, word in enumerate(unique_words):
  vector = np.zeros(len(unique_words))
  vector[i] = 1
  dict_one_hot[word] = torch.from_numpy(vector)
dict_one_hot

{'!': tensor([1., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64),
 '!!': tensor([0., 1., 0.,  ..., 0., 0., 0.], dtype=torch.float64),
 '!!!': tensor([0., 0., 1.,  ..., 0., 0., 0.], dtype=torch.float64),
 '!!!!': tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64),
 '!!!!!': tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64),
 '!!!!!!!!': tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64),
 '!!!!؟': tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64),
 '!!!برای': tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64),
 '!!!😂😂😂😂مگه': tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64),
 '!Say': tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64),
 '!؟': tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64),
 '!بگو': tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64),
 '!🤦🏻\u200d♀️🤦🏻\u200d♀️': tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64),
 '"': tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64),

In [ ]:
find_k_nearest_neighbors("زندگی", dict_one_hot, 10)

['زندگی',
 '\U0001faf6💪🌹🙏',
 '\U0001faf6🏽💔🕊',
 '\U0001faf6🏻',
 '\U0001faf6✨',
 '\U0001faf6✌️👊',
 '\U0001faf6#مهسا_امینی',
 '\U0001faf6',
 '\U0001faf1🏻\u200d\U0001faf2🏼🖤',
 '\U0001fae1']

#2. TF-IDF

In [ ]:
from collections import Counter

In [ ]:
Counter("من به آنجا من به همینجا".split())

Counter({'من': 2, 'به': 2, 'آنجا': 1, 'همینجا': 1})

In [ ]:
len(unique_words)

32771

In [ ]:
# 1. find the TF-IDF of all tweets.
# 2. choose one tweets randomly.
# 3. find 10 nearest tweets from chosen tweet.
num_of_sentences = len(pure_text_list)
word
for sentence in pure_text_list:
  words = sentence.split()
  words_frequency = Counter(words)
  for i, word in enumerate(words):
    word


In [ ]:
len(pure_text_list)

20000

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(pure_text_list)


In [ ]:
torch.from_numpy(X[3].toarray()[0])

tensor([0., 0., 0.,  ..., 0., 0., 0.], dtype=torch.float64)

In [ ]:
dict_tf_idf = {}
for vector, sentence in zip(X, pure_text_list):
  dict_tf_idf[sentence] = torch.from_numpy(vector.toarray()[0])

In [ ]:
list(dict_tf_idf.keys())[250]

'طوری ها می گویند «گشت ارشاد بهانه است» که انگار در دیگر زمینه های زندگی، هم سطح اتحادیه اروپا در حرکت هستیم. '

In [ ]:
find_k_nearest_neighbors(embedding_dict=dict_tf_idf, word=list(dict_tf_idf.keys())[250], k =10)

['طوری ها می گویند «گشت ارشاد بهانه است» که انگار در دیگر زمینه های زندگی، هم سطح اتحادیه اروپا در حرکت هستیم. ',
 'گویند چِهِل روز ولی انگار چِهِل سال عزادار تو بودیم... 🥀🖤🥀 ',
 'خمینی گجستک الان بود میگفت: طوری نکنید که طوری بشود که مردم فکر کنند طوری شده است\U0001fae0',
 'باید بدیمش گشت ارشاد 🤣',
 'هم هستیم',
 'داعش؟ نه ممنون گشت ارشاد داریم!',
 'قدم زدن تو\u200c خیابون بدون ترس از گشت ارشاد :)',
 'همه با هم در',
 'هستیم',
 'خیلی "و" ها داشته ایم ... دیگر بس است']

#3. Word2Vec

In [ ]:
# 1. train a word2vec model base on all tweets for each year.
import gensim
from gensim.models import Word2Vec

data = [sentence.split() for sentence in pure_text_list]
# 2. find 10 nearest words from "ولنتاین"
model1 = gensim.models.Word2Vec(data, min_count = 1, size = 100, window = 5)


In [ ]:
model1.wv.most_similar("زندگی")

[('میهن', 0.9983968138694763),
 ('زن،', 0.9976084232330322),
 ('زن', 0.9973310828208923),
 ('امید', 0.9944347143173218),
 ('زندگی،', 0.9943767786026001),
 ('آزادی', 0.9942494630813599),
 ('آزادی،به', 0.9939682483673096),
 ('ازادی', 0.9924623370170593),
 ('،زندگی', 0.9918760061264038),
 ('فردای', 0.9916279315948486)]

#4. Contextualized embedding

In [10]:
model_checkpoint = "HooshvareLab/bert-base-parsbert-uncased"

In [11]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.0 MB/s eta 0:00:00


In [12]:
from transformers import BertModel, BertTokenizer

model = BertModel.from_pretrained(model_checkpoint, output_hidden_states = True)
tokenizer = BertTokenizer.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
def bert_text_preparation(text, tokenizer):
  """
  Preprocesses text input in a way that BERT can interpret.
  """
  marked_text = "[CLS] " + text + " [SEP]"
  tokenized_text = tokenizer.tokenize(marked_text)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  segments_ids = [1]*len(indexed_tokens)
  # convert inputs to tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensor = torch.tensor([segments_ids])
  return tokenized_text, tokens_tensor, segments_tensor

def get_bert_embeddings(tokens_tensor, segments_tensor, model):
  """
  Obtains BERT embeddings for tokens.
  """
  # gradient calculation id disabled
  with torch.no_grad():
    # obtain hidden states
    outputs = model(tokens_tensor, segments_tensor)
    hidden_states = outputs[2]
  # concatenate the tensors for all layers
  # use "stack" to create new dimension in tensor
  token_embeddings = torch.stack(hidden_states, dim=0)
  # remove dimension 1, the "batches"
  token_embeddings = torch.squeeze(token_embeddings, dim=1)
  # swap dimensions 0 and 1 so we can loop over tokens
  token_embeddings = token_embeddings.permute(1,0,2)
  # intialized list to store embeddings
  token_vecs_sum = []
  # "token_embeddings" is a [Y x 12 x 768] tensor
  # where Y is the number of tokens in the sentence
  # loop over tokens in sentence
  for token in token_embeddings:
  # "token" is a [12 x 768] tensor
  # sum the vectors from the last four layers
    sum_vec = torch.sum(token[-4:], dim=0)
    token_vecs_sum.append(sum_vec)
  return token_vecs_sum

x = "زندگی"

tokenized_text, tokens_tensor, segments_tensor = bert_text_preparation(x, tokenizer)

x_vector = get_bert_embeddings(tokens_tensor, segments_tensor, model)

In [ ]:
x_vector = np.array([np.array(vector) for vector in x_vector])

In [ ]:
x_vector = x_vector.mean(axis=0)

In [ ]:
bert_dict = {}

bert_dict[x] = torch.from_numpy(x_vector)

counter = 0
for word in unique_words[:1000]:

  counter += 1
  print(counter, end = " ")

  tokenized_text, tokens_tensor, segments_tensor = bert_text_preparation(word, tokenizer)

  x_vector = get_bert_embeddings(tokens_tensor, segments_tensor, model)

  x_vector = np.array([np.array(vector) for vector in x_vector])

  x_vector = x_vector.mean(axis=0)

  bert_dict[word] = torch.from_numpy(x_vector)

find_k_nearest_neighbors(embedding_dict=bert_dict, word=x, k=10)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

['(مرگ',
 'زندگی',
 '(استبداد',
 '(هفتاد',
 '(خواهر',
 '(و',
 '(شصت',
 '(که',
 '(',
 '(همه']

In [ ]:
dataset.iloc[0]

{'input_ids': [2, 11368, 2093, 2268, 2945, 9558, 2179, 2945, 2063, 5528, 2584, 1, 23637, 1, 12299, 1, 33596, 21063, 1, 9664, 40315, 1, 33596, 21063, 56741, 1165, 4], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
df["Sentiment"].unique()

array(['negative', 'very negative', 'positive', 'no sentiment expressed',
       'very positive', 'mixed'], dtype=object)

In [14]:
def transform_labels(label):

    sentiment = label['Sentiment']
    num = 0
    if sentiment == 'very negative':
        num = 0
    elif sentiment == 'negative':
        num = 1
    elif sentiment == 'no sentiment expressed':
        num = 2
    elif sentiment == 'positive':
        num = 3
    elif sentiment == 'very positive':
        num = 4
    elif sentiment == 'mixed':
        num = 5

    label["Sentiment"] = {"labels" : num}

    return label

def tokenize_data(example):
    example["Text"] = tokenizer(example['Text'], padding='max_length')
    return example

dataset = df.iloc[:1000].apply(tokenize_data,axis="columns")

remove_columns = list(df.columns)

remove_columns.remove('Text')
remove_columns.remove("Sentiment")
dataset = dataset.apply(transform_labels, axis="columns")
dataset

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.


,Unnamed: 0,Datetime,Text,PureText,Language,Sentiment,Date
0,0,2022-09-22 09:14:35+00:00,"[input_ids, token_type_ids, attention_mask]",بنشین تا شود نقش فال ما نقش هم‌ فردا شدن,fa,{'labels': 1},2022-09-22
1,1,2022-10-06 01:44:55+00:00,"[input_ids, token_type_ids, attention_mask]",این گوزو رو کی گردن میگیره؟؟ دچار زوال عقل شده...,fa,{'labels': 0},2022-10-06
2,2,2022-09-22 15:12:28+00:00,"[input_ids, token_type_ids, attention_mask]",برای ایران، برای مهسا.,fa,{'labels': 3},2022-09-22
3,3,2022-09-22 09:35:50+00:00,"[input_ids, token_type_ids, attention_mask]",مرگ بر دیکتاتور,fa,{'labels': 0},2022-09-22
4,4,2022-09-22 01:31:25+00:00,"[input_ids, token_type_ids, attention_mask]",نذاریم خونشون پایمال شه.‌‌.‌‌.,fa,{'labels': 1},2022-09-22
...,...,...,...,...,...,...,...
995,995,2022-09-23 12:58:01+00:00,"[input_ids, token_type_ids, attention_mask]",هرچی میتونین هشتگ بزنین,fa,{'labels': 2},2022-09-23
996,996,2022-10-15 23:11:51+00:00,"[input_ids, token_type_ids, attention_mask]",من گلو درد ام نمیتونم گریه کنم بچه دو ساله ام ...,fa,{'labels': 1},2022-10-15
997,997,2022-09-25 08:29:38+00:00,"[input_ids, token_type_ids, attention_mask]",ولی من عاشققق متحد شدنمونم دختر و پسر مرد و زن...,fa,{'labels': 3},2022-09-25
998,998,2022-09-22 12:58:49+00:00,"[input_ids, token_type_ids, attention_mask]",به زودی باید برن تو همون استادیوم با اشک چشم س...,fa,{'labels': 1},2022-09-22


In [15]:
dataset = dataset.drop(columns=remove_columns)

In [16]:
dataset

,Text,Sentiment
0,"[input_ids, token_type_ids, attention_mask]",{'labels': 1}
1,"[input_ids, token_type_ids, attention_mask]",{'labels': 0}
2,"[input_ids, token_type_ids, attention_mask]",{'labels': 3}
3,"[input_ids, token_type_ids, attention_mask]",{'labels': 0}
4,"[input_ids, token_type_ids, attention_mask]",{'labels': 1}
...,...,...
995,"[input_ids, token_type_ids, attention_mask]",{'labels': 2}
996,"[input_ids, token_type_ids, attention_mask]",{'labels': 1}
997,"[input_ids, token_type_ids, attention_mask]",{'labels': 3}
998,"[input_ids, token_type_ids, attention_mask]",{'labels': 1}


In [17]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer", num_train_epochs=3)

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [18]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=dataset, eval_dataset=dataset
)

In [19]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 375
  Number of trainable parameters = 162841344


KeyError: ignored

In [ ]:

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer", num_train_epochs=3)

from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=6)


from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=dataset, eval_dataset=dataset
)

trainer.train()

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--HooshvareLab--bert-base-parsbert-uncased/snapshots/d73a0e2c7492c33bd5819bcdb23eba207404dd19/config.json
Model config BertConfig {
  "_name_or_path": "HooshvareLab/bert-base-parsbert-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}

loading file vocab.txt from cache at /root/.cache/huggingface/hu

KeyError: ignored